# Action Plan

- Download the data
    - Place the dataset in data/stroke folder !!! (dont forget to do this)
    - Load the data into a pandas dataframe?
- Preprocess the signals
    - Remove the noise
    - Normalize the data
- Feature extraction
    - Extract features from the signals
- Train the model
    - Train the model using the extracted features
- Evaluate the model
    - Evaluate the model using the test data


# Data Loading
- Load the data into a pandas dataframe from matlab file

In [ ]:
# Load the data loadmat
from scipy.io import loadmat

# Load the data
data_dict = {
    "Patient_1" : {
        "Pre":{
            "Train": loadmat('data/stroke/P1_pre_training.mat'),
            "Test": loadmat('data/stroke/P1_pre_test.mat')
        },
        "Post":{
            "Train": loadmat('data/stroke/P1_post_training.mat'),
            "Test": loadmat('data/stroke/P1_post_test.mat')
        }
    },
    "Patient_2" : {
        "Pre":{
            "Train": loadmat('data/stroke/P2_pre_training.mat'),
            "Test": loadmat('data/stroke/P2_pre_test.mat')
        },
        "Post":{
            "Train": loadmat('data/stroke/P2_post_training.mat'),
            "Test": loadmat('data/stroke/P2_post_test.mat')
        }
    },
    "Patient_3" : {
        "Pre":{
            "Train": loadmat('data/stroke/P3_pre_training.mat'),
            "Test": loadmat('data/stroke/P3_pre_test.mat')
        },
        "Post":{
            "Train": loadmat('data/stroke/P3_post_training.mat'),
            "Test": loadmat('data/stroke/P3_post_test.mat')
        }
    },
}



In [ ]:
# Trying to know the content of the data
print("Data keys:")

print(data_dict["Patient_1"]["Pre"]["Train"].keys())
'''fs - sampling rate Hz
y - EEG data (n_channels x n_samples)
trig - trigger data (+1 left hand MI , -1 right hand MI)
'''

print("F sampling rate:")
print(data_dict["Patient_1"]["Pre"]["Train"]["fs"])
print("EEG data shape (samples, channels):")
print(data_dict["Patient_1"]["Pre"]["Train"]["y"].shape)
print("Trigger data shape:")
print(data_dict["Patient_1"]["Pre"]["Train"]["trig"].shape)


'''Separate epochs for left and right hand MI'''
from scripts import load_mne

# data = load_mne(data_dict["Patient_1"]["Pre"]["Train"]["y"], data_dict["Patient_1"]["Pre"]["Train"]["fs"], data_dict["Patient_1"]["Pre"]["Train"]["trig"])
mne_data_dict = load_mne.mne_load_data(data_dict=data_dict)

# divide the eeg into epochs, each epoch is 8 seconds long (trigger is at 2 seconds, at 3.5 seconds )
# One session was composed by 240 MI repetitions on both hands, divided in 3 runs of 80 trials each.



In [ ]:
import numpy as np
# plot eeg and triggers for the first 10000 samples
import matplotlib.pyplot as plt

eeg_trigs = data_dict["Patient_1"]["Pre"]["Train"]["trig"][0, :]
out_trigs = np.zeros_like(eeg_trigs)
# find the triggers
idx_array = []
for i in range(len(eeg_trigs)):
    if (eeg_trigs[i] != eeg_trigs[i-1]):
        out_trigs.append(eeg_trigs[i])
        idx_array.append(i)


# Define the time axis and plot the data
plt.figure(figsize=(10, 5))
start_samples = 256 * 60 * 6
width_samples = 256 * 60 * 2
end_samples = width_samples+ start_samples
data = data_dict["Patient_1"]["Pre"]["Train"]["y"]
triggers = data_dict["Patient_1"]["Pre"]["Train"]["trig"]

print(data.shape)
time_axis = np.arange(start_samples, end_samples) / data_dict["Patient_1"]["Pre"]["Train"]["fs"][0][0]
plt.plot(time_axis, data[start_samples:end_samples, 0], label='Channel 1')
# plt.plot(time_axis, data[start_samples:end_samples, 1], label='Channel 2')

plt.plot(time_axis, triggers[start_samples:end_samples], label='Trigger')

triggerd = np.zeros_like(triggers, dtype=int)
triggerd[1:] = ((triggers[1:]-triggers[:-1])!=0) * (triggers[1:]!=0)
triggerd[triggerd!=0] = triggers[triggerd!=0]
events = np.column_stack((np.argwhere(triggerd)[:,0], np.zeros(sum(triggerd!=0), dtype=int),triggerd[triggerd!=0]))
# print(events)

plt.plot(time_axis, triggerd[start_samples:end_samples], label='Triggerdiff')

plt.xlabel('Seconds')
plt.legend()

plt.show()


In [ ]:
# mne plots
import numpy as np
import matplotlib.pyplot as plt

# spectrogram, event related spectral perturbation (ERSP) and inter-trial coherence (ITC)

# SELECTING ONE PATIENT

patient_epochs = mne_data_dict["Patient_1"]["Pre"]["Train"]
print(patient_epochs)

# plot the spectrogram of the epoch using mne

epoch.plot_psd(fmin=2., fmax=40., average=True, spatial_colors=False)





# Preprocessing

lowpass = 0.1 # Hz
highpass = 50 # Hz
